In [1]:
import pandas as pd
import numpy as np

from sklearn.preprocessing import MinMaxScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

In [2]:
df = pd.read_csv("02_Test_Data_Set/sales_pos.csv")

In [3]:
df.shape

(550068, 11)

### Q1.

In [5]:
df_q1 = df[["prod", "job", "purchase"]].reset_index(drop = True)
df_q1.head(2)

,prod,job,purchase
0,P00069042,10,8370
1,P00248942,10,15200


In [7]:
df_q1_agg = df_q1.groupby("prod")["purchase"].sum()
df_q1_agg.head(2)

prod
P00000142    12837476
P00000242     3967496
Name: purchase, dtype: int64

In [9]:
df_q1_agg[df_q1_agg == df_q1_agg.max()]

prod
P00025442    27995166
Name: purchase, dtype: int64

In [8]:
df_q1.groupby("prod")["purchase"].sum().idxmax()

'P00025442'

In [10]:
val_prod = df_q1.groupby("prod")["purchase"].sum().idxmax()
val_prod

'P00025442'

In [11]:
df_q1_sub = df_q1.loc[df_q1["prod"] == val_prod, ]
df_q1_sub.head(2)

,prod,job,purchase
667,P00025442,17,19706
749,P00025442,7,15212


In [13]:
df_q1_sub["job"].value_counts().idxmax()

4

In [15]:
df.loc[df["prod"] == df.groupby("prod")["purchase"].sum().idxmax(), "job"].value_counts().idxmax()

4

### Q2.

In [19]:
df_q2_test = df.loc[df["user"] == 1, ["user", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2_test = df_q2_test.reset_index(drop = True).fillna(0)
df_q2_test

,user,prod_cat1,prod_cat2,prod_cat3
0,1,3,0.0,0.0
1,1,1,6.0,14.0
2,1,12,0.0,0.0
3,1,12,14.0,0.0
4,1,2,4.0,8.0
5,1,4,8.0,9.0
6,1,1,2.0,5.0
7,1,3,4.0,12.0
8,1,5,15.0,0.0
9,1,5,0.0,0.0


In [22]:
df_q2_test.drop_duplicates().shape

(21, 4)

In [23]:
df_q2_test["prod_cat1"] = df_q2_test["prod_cat1"].astype("int")
df_q2_test["prod_cat2"] = df_q2_test["prod_cat2"].astype("int")
df_q2_test["prod_cat3"] = df_q2_test["prod_cat3"].astype("int")

In [24]:
df_q2_test.head(2)

,user,prod_cat1,prod_cat2,prod_cat3
0,1,3,0,0
1,1,1,6,14


In [25]:
df_q2_test["prod_cat1"] = df_q2_test["prod_cat1"].astype("str")
df_q2_test["prod_cat2"] = df_q2_test["prod_cat2"].astype("str")
df_q2_test["prod_cat3"] = df_q2_test["prod_cat3"].astype("str")

In [26]:
df_q2_test.head(2)

,user,prod_cat1,prod_cat2,prod_cat3
0,1,3,0,0
1,1,1,6,14


In [27]:
df_q2_test.dtypes

user          int64
prod_cat1    object
prod_cat2    object
prod_cat3    object
dtype: object

In [28]:
df_q2_test["prod_cat"] = df_q2_test["prod_cat1"] + "-" + df_q2_test["prod_cat2"] + "-" + df_q2_test["prod_cat3"]
df_q2_test.head(2)

,user,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,1,3,0,0,3-0-0
1,1,1,6,14,1-6-14


In [33]:
df_q2_test["prod_catt"] = df_q2_test.loc[:, "prod_cat1":"prod_cat3"].apply(lambda x: x.str.cat(sep = "-"), axis = 1)
df_q2_test.head(2)

,user,prod_cat1,prod_cat2,prod_cat3,prod_cat,prod_catt
0,1,3,0,0,3-0-0,3-0-0
1,1,1,6,14,1-6-14,1-6-14


In [41]:
df_q2_test["prod_cat"].nunique()

21

In [31]:
ser = pd.Series(["a", "b", "c"])
ser.str.cat(sep = "-")

'a-b-c'

In [34]:
df["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [37]:
df_q2 = df.loc[df["age_group"] == "26-35", ]
df_q2 = df_q2[["user", "marital", "prod_cat1", "prod_cat2", "prod_cat3"]]
df_q2 = df_q2.reset_index(drop = True)
df_q2.head(2)

,user,marital,prod_cat1,prod_cat2,prod_cat3
0,3,0,1,2.0,NaN
1,5,1,8,NaN,NaN


In [38]:
df_q2.isna().sum()

user              0
marital           0
prod_cat1         0
prod_cat2     69427
prod_cat3    152645
dtype: int64

In [40]:
df_q2 = df_q2.fillna(0)
df_q2["prod_cat1"] = df_q2["prod_cat1"].astype("int").astype("str")
df_q2["prod_cat2"] = df_q2["prod_cat2"].astype("int").astype("str")
df_q2["prod_cat3"] = df_q2["prod_cat3"].astype("int").astype("str")
df_q2["prod_cat" ] = df_q2.loc[:, "prod_cat1":"prod_cat3"].apply(lambda x: x.str.cat(sep = "-"), 
                                                                 axis = 1)
df_q2.head()

,user,marital,prod_cat1,prod_cat2,prod_cat3,prod_cat
0,3,0,1,2,0,1-2-0
1,5,1,8,0,0,8-0-0
2,5,1,5,11,0,5-11-0
3,5,1,8,0,0,8-0-0
4,5,1,8,0,0,8-0-0


In [43]:
df_q2_m0 = df_q2.loc[df_q2["marital"] == 0, ]
df_q2_m1 = df_q2.loc[df_q2["marital"] == 1, ]

In [46]:
mean_m0 = df_q2_m0.groupby("user")["prod_cat"].nunique().mean()
mean_m1 = df_q2_m1.groupby("user")["prod_cat"].nunique().mean()
round(abs(mean_m0 - mean_m1), 2)

0.13

In [49]:
ser = pd.Series([1, 3, 3, 5, 5, 5, 7])
ser.unique()

array([1, 3, 5, 7], dtype=int64)

In [50]:
len(ser.unique())

4

In [51]:
ser.nunique() # number of unique value(element)

4

### Q3.

In [52]:
df.head(2)

,user,prod,gender,age_group,job,city,marital,prod_cat1,prod_cat2,prod_cat3,purchase
0,1,P00069042,F,0-17,10,A,0,3,NaN,NaN,8370
1,1,P00248942,F,0-17,10,A,0,1,6.0,14.0,15200


In [53]:
df["user"].nunique()

5891

In [56]:
df_user = df[["user", "gender", "age_group", "job", "city", "marital"]].drop_duplicates()
df_user = df_user.reset_index(drop = True)
df_user.head()

,user,gender,age_group,job,city,marital
0,1,F,0-17,10,A,0
1,2,M,55+,16,C,0
2,3,M,26-35,15,A,0
3,4,M,46-50,7,B,1
4,5,M,26-35,20,A,1


In [57]:
len(df_user)

5891

In [61]:
df_user_agg1 = df.groupby("user")["prod"].nunique().reset_index()
df_user_agg2 = df.groupby("user")["purchase"].sum().reset_index()
df_user_agg1.head(2)

,user,prod
0,1,35
1,2,77


In [70]:
df_join = df_user.merge(df_user_agg1, left_on = "user", right_on = "user")
df_join = df_join.merge(df_user_agg2, left_on = "user", right_on = "user")
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase
0,1,F,0-17,10,A,0,35,334093
1,2,M,55+,16,C,0,77,810472


In [71]:
df_join["gender"] = (df_join["gender"] == "M") + 0

In [72]:
df_join["age_group"].unique()

array(['0-17', '55+', '26-35', '46-50', '51-55', '36-45', '18-25'],
      dtype=object)

In [73]:
df_join["age_g"] = df_join["age_group"].replace({"0-17": 0, "18-25": 1, "26-35": 2,
                                                 "36-45": 3, "46-50": 4, "51-55": 5,
                                                 "55+": 6})
df_join.head(2)

,user,gender,age_group,job,city,marital,prod,purchase,age_g
0,1,0,0-17,10,A,0,35,334093,0
1,2,1,55+,16,C,0,77,810472,6


In [74]:
pd.crosstab(df_join["age_group"], df_join["age_g"])

age_g,0,1,2,3,4,5,6
age_group,,,,,,,
0-17,218,0,0,0,0,0,0
18-25,0,1069,0,0,0,0,0
26-35,0,0,2053,0,0,0,0
36-45,0,0,0,1167,0,0,0
46-50,0,0,0,0,531,0,0
51-55,0,0,0,0,0,481,0
55+,0,0,0,0,0,0,372


In [75]:
df_join = df_join.drop(["user", "age_group"], axis = 1)
df_join_dum = pd.get_dummies(df_join, columns = ["job", "city"])
df_join_dum.head(2)

,gender,marital,prod,purchase,age_g,job_0,job_1,job_2,job_3,job_4,...,job_14,job_15,job_16,job_17,job_18,job_19,job_20,city_A,city_B,city_C
0,0,0,35,334093,0,0,0,0,0,0,...,0,0,0,0,0,0,0,1,0,0
1,1,0,77,810472,6,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,1


In [78]:
df_join_dum_nor = MinMaxScaler().fit_transform(df_join_dum)

In [79]:
df_join_dum_nor[:1, ]

array([[0.        , 0.        , 0.02843137, 0.02739807, 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        1.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 0.        , 0.        , 0.        , 0.        ,
        0.        , 1.        , 0.        , 0.        ]])

In [81]:
model_kmeans = KMeans(n_clusters = 7, random_state = 123)
model_kmeans.fit(df_join_dum_nor)

KMeans(n_clusters=7, random_state=123)

In [83]:
round(silhouette_score(df_join_dum_nor, labels = model_kmeans.labels_), 2)

0.18